In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from matplotlib.pyplot import rc_context

import sys
import warnings

import scvi
import anndata
import matplotlib.pyplot as plt

Global seed set to 0


In [2]:
#set up directory
large_root = r"../03-Out"
results_file = large_root + "/results_file.h5ad"

In [3]:
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 1
sc.logging.print_header()

scanpy==1.9.2 anndata==0.8.0 umap==0.5.3 numpy==1.23.5 scipy==1.10.0 pandas==1.5.3 scikit-learn==1.2.1 statsmodels==0.13.5 python-igraph==0.10.4 pynndescent==0.5.4


In [6]:
# index2 = pd.read_csv("../01-data/GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt", delimiter="\t",skiprows=[0], nrows=0).T

url = "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE120575&format=file&file=GSE120575%5FSade%5FFeldman%5Fmelanoma%5Fsingle%5Fcells%5FTPM%5FGEO%2Etxt%2Egz"
url2 = pd.read_csv(url, delimiter="\t",skiprows=[0], nrows=0).T
import gzip
with gzip.open(url2, 'rb') as f:
    file_content = f.read()
index2 = pd.read_csv(file_content, delimiter="\t",skiprows=[0], nrows=0).T


print(index2.shape)
index2 = index2.iloc[1:,:]
print(index2.shape)
index2 = index2.reset_index()
index2.rename(columns = {'index':'index2'}, inplace = True)

index2.head()
# adata = adata.iloc[1:,:].copy()
# adata.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [ ]:
index3 = pd.read_csv(url, delimiter="\t",skiprows=[1], nrows=0).T

print(index3.shape)
index3 = index3.iloc[1:,:]
print(index3.shape)
index3 = index3.reset_index()
index3.rename(columns = {'index':'index3'}, inplace = True)

index3.head()
# adata = adata.iloc[1:,:].copy()
# adata.head()

In [ ]:
index3

In [ ]:
index3_list = index3.index3.to_list()
len(index3_list)
index3_list = index3_list[:-1]
len(index3_list)

In [ ]:
combo_index = pd.concat([index2, index3], axis="columns")
combo_index = combo_index.iloc[:-1,:]

In [ ]:
combo_index['full_sample_name'] = combo_index['index3'] + "-" +combo_index['index2']
combo_index = combo_index.drop('index2', axis = 1)
combo_index = combo_index.drop('index3', axis = 1)
combo_index = combo_index.set_index("full_sample_name")
combo_index.head()

In [ ]:
cell_index = []
for i in range(16290):
    cell_index.append('Sample '+str(i+1))
cell_index
len(cell_index)

combo_index2 = combo_index.reset_index()
combo_index2['sample_name'] = cell_index
combo_index2 = combo_index2.set_index('sample_name')
combo_index2

In [ ]:
combo_index.shape

In [ ]:
cell_names_list = combo_index.index.to_list()
print(len(cell_names_list))
# cell_names_list = cell_names_list[:-1]
print(len(cell_names_list))
cell_names_list

In [ ]:
gene_names = pd.read_csv("../01-data/GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt", delimiter="\t", usecols=[0])

gene_names.rename(columns = {'Unnamed: 0':'gene_symbols'}, inplace = True)
gene_names = gene_names.iloc[1:,:]

gene_names_list = gene_names.gene_symbols.to_list()
print(gene_names_list[0])

# gene_names_list = gene_names_list[1:]
# gene_names_list

print(gene_names.shape)

In [ ]:
print(len(gene_names_list))

In [ ]:
# read data
adata = pd.read_csv("../01-data/GSE120575_Sade_Feldman_melanoma_single_cells_TPM_GEO.txt", delimiter="\t",skiprows=[1], usecols=index3_list, header=0).T

print(adata.shape)

In [ ]:
bdata = adata.to_numpy()
print(bdata.shape)
bdata

In [ ]:
# create sparse matrix using csr_matrix()
  
# Import required package
from scipy.sparse import csr_matrix
from scipy import sparse
  
# creating sparse matrix
sparseMatrix = sparse.csr_matrix(bdata)
              
# print the sparse matrix
print(sparseMatrix)

In [ ]:
type(sparseMatrix)
sparse.isspmatrix(sparseMatrix)

In [ ]:
import scipy.io as sio
sio.mmwrite("../01-data/matrix/matrix.mtx",sparseMatrix)

In [ ]:
gene_names

In [ ]:
gene_names.to_csv('../01-data/matrix/genes.tsv', index=False)

In [ ]:
combo_index2.to_csv('../01-data/matrix/barcodes.tsv')

In [ ]:


# index2.rename(columns = {'index':'index2'}, inplace = True)
